In [6]:
#SERINE


import os
from Bio.PDB import PDBParser, PDBIO, Select, Atom

# Directory containing PDB files
pdb_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
os.makedirs(output_dir, exist_ok=True)

# List of PDB files
pdb_files = ["original_humansec.pdb"]

# List of mutations (residue_number, new_residue)
mutations = {
    47: "SER",  
}

class MutateSelect(Select):
    def __init__(self, mutations):
        self.mutations = mutations

    def accept_residue(self, residue):
        res_id = residue.get_id()[1]
        if res_id in self.mutations:
            new_residue = self.mutations[res_id]
            residue.resname = new_residue

        # Add missing atoms based on residue type
        if residue.resname == "SER" and res_id == 47:
            self.add_ser_atoms(residue)

        return True

    def add_ser_atoms(self, residue):
        """Add missing atoms for SER residue at position 47."""
        atom_names = [atom.get_name() for atom in residue]
        atoms_to_add = {
            "OG": [20.154, 42.000, 18.200],  # Example coordinates for OG atom
            "H01": [22.206, 46.016, 18.566],
            "H02": [22.024, 45.430, 16.271],
            "H03": [20.374, 45.802, 16.750],
            "H04": [20.667, 43.270, 17.413],
            "H10": [22.723, 43.306, 18.024],
        }
        for atom_name, coords in atoms_to_add.items():
            if atom_name not in atom_names:
                self.add_atom(residue, atom_name, coords, "O" if atom_name == "OG" else "H")

    def add_atom(self, residue, atom_name, coords, element):
        """Helper function to add an atom to a residue."""
        new_atom = Atom.Atom(atom_name, coords, 1.0, 1.0, " ", atom_name, residue.get_id()[1], element=element)
        residue.add(new_atom)
        print(f"Added atom {atom_name} to residue {residue.resname} {residue.get_id()[1]} at coordinates {coords}")

# Function to mutate residues in a PDB file
def mutate_pdb(pdb_file, output_file, mutations):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    
    io = PDBIO()
    io.set_structure(structure)
    
    # Save the mutated structure without removing any atoms
    io.save(output_file, MutateSelect(mutations))
    print(f"Mutated PDB saved to {output_file}")

# Apply mutations to all PDB files
for pdb_file in pdb_files:
    pdb_path = os.path.join(pdb_dir, pdb_file)
    output_path = os.path.join(output_dir, "A47S.pdb")
    mutate_pdb(pdb_path, output_path, mutations)

print("All mutations completed successfully.")


Added atom OG to residue SER 47 at coordinates [20.154, 42.0, 18.2]
Added atom H10 to residue SER 47 at coordinates [22.723, 43.306, 18.024]
Mutated PDB saved to /home/hp/nayanika/github/GPX6/prep_structures/A47S.pdb
All mutations completed successfully.


In [7]:
#Phenylalanine

import os
from Bio.PDB import PDBParser, PDBIO, Select, Atom

# Directory containing PDB files
pdb_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
os.makedirs(output_dir, exist_ok=True)

# List of PDB files
pdb_files = ["original_humansec.pdb"]

# List of mutations (residue_number, new_residue)
mutations = {
    48: "PHE",  
}

class MutateSelect(Select):
    def __init__(self, mutations):
        self.mutations = mutations

    def accept_residue(self, residue):
        res_id = residue.get_id()[1]
        if res_id in self.mutations:
            new_residue = self.mutations[res_id]
            residue.resname = new_residue

            # Remove OH atom if it's PHE
            if new_residue == "PHE":
                self.remove_oh_atom(residue)

        return True

    def remove_oh_atom(self, residue):
        """Remove the OH atom from the residue if present (for TYR -> PHE mutation)."""
        if "OH" in [atom.get_name() for atom in residue]:
            oh_atom = residue["OH"]
            residue.detach_child(oh_atom.get_id())
            print(f"Removed OH atom from residue {residue.resname} {residue.get_id()[1]}")

    def add_phe_atoms(self, residue):
        """Add missing atoms for PHE residue at position 104."""
        atom_names = [atom.get_name() for atom in residue]
        atoms_to_add = {
            "H01": [41.186, 43.018, 14.811],  # Example coordinates for PHE hydrogen atoms
            "H02": [41.037, 40.368, 16.374],
            "H03": [42.571, 40.677, 14.128],
            "H04": [40.223, 40.608, 14.766],
            "H05": [39.798, 40.306, 18.099],
            "H06": [38.457, 42.702, 14.794],
            "H07": [37.652, 40.570, 19.258],
            "H08": [36.302, 43.120, 16.044],
            "H09": [36.147, 42.488, 18.520],
        }
        for atom_name, coords in atoms_to_add.items():
            if atom_name not in atom_names:
                self.add_atom(residue, atom_name, coords, "H")

    def add_atom(self, residue, atom_name, coords, element):
        """Helper function to add an atom to a residue."""
        new_atom = Atom.Atom(atom_name, coords, 1.0, 1.0, " ", atom_name, residue.get_id()[1], element=element)
        residue.add(new_atom)
        print(f"Added atom {atom_name} to residue {residue.resname} {residue.get_id()[1]} at coordinates {coords}")

# Function to mutate residues in a PDB file
def mutate_pdb(pdb_file, output_file, mutations):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    
    io = PDBIO()
    io.set_structure(structure)
    
    # Save the mutated structure without removing any atoms
    io.save(output_file, MutateSelect(mutations))
    print(f"Mutated PDB saved to {output_file}")

# Apply mutations to all PDB files
for pdb_file in pdb_files:
    pdb_path = os.path.join(pdb_dir, pdb_file)
    output_path = os.path.join(output_dir, "Y48F.pdb")
    mutate_pdb(pdb_path, output_path, mutations)

print("All mutations completed successfully.")


Removed OH atom from residue PHE 48
Mutated PDB saved to /home/hp/nayanika/github/GPX6/prep_structures/Y48F.pdb
All mutations completed successfully.


In [14]:
import os
from Bio.PDB import PDBParser, PDBIO, Select, Atom

# Directory containing PDB files
pdb_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
os.makedirs(output_dir, exist_ok=True)

# List of PDB files
pdb_files = ["original_humansec.pdb"]

# List of mutations (residue_number, new_residue)
mutations = {
    52: "THR",  
}

class MutateSelect(Select):
    def __init__(self, mutations):
        self.mutations = mutations

    def accept_residue(self, residue):
        res_id = residue.get_id()[1]
        if res_id in self.mutations:
            new_residue = self.mutations[res_id]
            residue.resname = new_residue

        # Add missing atoms based on residue type
        if residue.resname == "THR" and res_id == 52:
            self.add_thr_atoms(residue)

        return True

    def add_thr_atoms(self, residue):
        """Add missing atoms for THR residue at position 52."""
        atom_names = [atom.get_name() for atom in residue]
        atoms_to_add = {
            "OG1": [30.497, 28.982, 41.315],  # Coordinates for OG1 atom
            "CG2": [28.899, 29.646, 42.880],  # Coordinates for CG2 atom
            "H01": [29.889, 31.153, 39.451],  # Example coordinates for H atoms
            "H02": [30.457, 30.908, 42.056],
            "H03": [27.928, 29.344, 42.488],
        }
        for atom_name, coords in atoms_to_add.items():
            if atom_name not in atom_names:
                self.add_atom(residue, atom_name, coords, "O" if "OG" in atom_name else "H")

    def add_atom(self, residue, atom_name, coords, element):
        """Helper function to add an atom to a residue."""
        new_atom = Atom.Atom(atom_name, coords, 1.0, 1.0, " ", atom_name, residue.get_id()[1], element=element)
        residue.add(new_atom)
        print(f"Added atom {atom_name} to residue {residue.resname} {residue.get_id()[1]} at coordinates {coords}")

# Function to mutate residues in a PDB file
def mutate_pdb(pdb_file, output_file, mutations):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    
    io = PDBIO()
    io.set_structure(structure)
    
    # Save the mutated structure without removing any atoms
    io.save(output_file, MutateSelect(mutations))
    print(f"Mutated PDB saved to {output_file}")

# Apply mutations to all PDB files
for pdb_file in pdb_files:
    pdb_path = os.path.join(pdb_dir, pdb_file)
    output_path = os.path.join(output_dir, "A52T.pdb")
    mutate_pdb(pdb_path, output_path, mutations)

print("All mutations completed successfully.")


Added atom OG1 to residue THR 54 at coordinates [30.497, 28.982, 41.315]
Added atom CG2 to residue THR 54 at coordinates [28.899, 29.646, 42.88]
Mutated PDB saved to /home/hp/nayanika/github/GPX6/prep_structures/Q54T.pdb
All mutations completed successfully.
